In [ ]:
# default_exp command_grabber

# Command grabber
> Script to run any command in command-line that grabbs the output and calls a callback for each printed line.

In [ ]:
#export
import threading
import time
import os, sys, select, subprocess

In [ ]:
#export
class CommandGrabber:
    def __init__(self, subprocess_cmd, line_callback=None):
        # subprocess_cmd: list of strings, the same as the subprosess command would take to execute it.
        # line_callback: function callback that takes the string as an input to handle it.
        super().__init__()
        self.subprocess_cmd = subprocess_cmd
        self.line_callback = line_callback
        self.running = False
        self.t = threading.Thread(target=self.run)
        self.t.start()
        
    def run(self):
        p1 = subprocess.Popen(self.subprocess_cmd, stdout=subprocess.PIPE)
        self.running=True
        while self.running:
            rlist, wlist, xlist = select.select([p1.stdout], [], [])
            time.sleep(0.1)
            for stdout in rlist:
                text = os.read(stdout.fileno(), 1024)  
                if self.line_callback is not None:
                    text = text.decode("utf-8")
                    self.line_callback(text)
        try:
            p1.kill()
            print("Successfully stopped subprocess!")
        except subprocess.CalledProcessError:
            print(f"Unable to kill subprocess (pid={p1.pid}) successfully...")
            return False
        return True
    
    def stop(self):
        self.running= False
        
    def wait_until_finished(self):
        self.t.join()

# Example 
we send the command to print the time every 2 seconds and let the "line_callback" function receive the line and check that the string seems correct.

In [ ]:
num_calls=0
def line_callback(line):
    
    global num_calls
    ret='no'
    if 'CEST' in line:
        num_calls+=1
        ret='ok'
    
    print(f"{line}: {ret}")

In [ ]:
num_calls=0
args = ['sh', '-c', 'while true; do date; sleep 2; done']
a = CommandGrabber(args, line_callback)
time.sleep(9)
a.stop()
#assert num_calls==5,f"num_calls was:{num_calls}" # does not seem to work in github by some reason

lör  3 jul 2021 00:44:43 CEST
: ok
lör  3 jul 2021 00:44:45 CEST
: ok
lör  3 jul 2021 00:44:47 CEST
: ok
lör  3 jul 2021 00:44:49 CEST
: ok
lör  3 jul 2021 00:44:51 CEST
: ok
lör  3 jul 2021 00:44:53 CEST
: ok
Successfully stopped subprocess!


In [ ]:
from nbdev.export import *
notebook2script()